In [91]:
import numpy as np
import matplotlib.pyplot as plt

### Creating a Neuron

In [92]:
def activation(x):
    return 1/(1 + np.exp(-x)) 
    # the activation function here is a sigmoid function, but we can even use functions like the binary step or linear function

In [93]:
class neuron:
    def __init__(self, weight, bias):
        self.weight = weight
        self.bias = bias
    def feedforward(self, inp):
        output = np.dot(self.weight, inp) + self.bias
        return activation(output)

In [94]:
# parameters
weight = np.array([1, 2])
bias = 5
n = neuron(weight, bias)

inp = np.array([3, 4])
print(n.feedforward(inp)) # neuron output

0.9999998874648379


### Feedforward Neural Network

In [95]:
class network:
    def __init__(self):
        weight = np.array([1, 2])
        bias = 5

        self.h1 = neuron(weight, bias)
        self.h2 = neuron(weight, bias)
        self.o1 = neuron(weight, bias)

    def feedforward(self, x):
        out_h1 = self.h1.feedforward(x)
        out_h2 = self.h2.feedforward(x)
        out_o1 = self.o1.feedforward(np.array([out_h1, out_h2]))

        return out_o1

neural_network = network()
x = np.array([3, 4])
print(neural_network.feedforward(x))

0.9996646497563555


### Mean Squared Loss

In [96]:
def mse(y_exp, y_act):
    mean_squared_error = ((y_act - y_exp)**2).mean()
    return mean_squared_error

y_act = np.array([1, 0, 0, 1])
y_exp = np.array([0, 0, 0, 0])

print(mse(y_act, y_exp))

0.5


### Applying Stochastic Gradient Descent to Minimise Loss

$w_1 \leftarrow w_1 - \eta \frac{\partial L}{\partial w_1}$ where $\eta$ is the learning rate (how fast we train)

This is simple trial network to predict sex based on height and weight

In [97]:
def der_act(x):
    y = activation(x)
    return y * (1 - y)

In [99]:
class new_network:
    def __init__(self):
        self.w1 = np.random.normal() # random samples from a Gaussian distribution
        self.w2 = np.random.normal()
        self.w3 = np.random.normal()
        self.w4 = np.random.normal()
        self.w5 = np.random.normal()
        self.w6 = np.random.normal()

        self.b1 = np.random.normal() # for h1
        self.b2 = np.random.normal() # for h2
        self.b3 = np.random.normal() # for o1

    def ff(self, x):
        h1 = activation(self.w1 * x[0] + self.w2 * x[1] + self.b1) # numpy array with two elements
        h2 = activation(self.w3 * x[0] + self.w4 * x[1] + self.b2)
        o1 = activation(self.w5 * h1 + self.w6 * h2 + self.b3)
        return o1

    def train(self, data, all_y_act):
        learn_rate = 0.1
        epochs = 1000 # number of times to loop through entire dataset
        for i in range(epochs):
            for x, y_true in zip(data, all_y_act):
                sum_h1 = self.w1 * x[0] + self.w2 * x[1] + self.b1
                h1 = activation(sum_h1)
                sum_h2 = self.w3 * x[0] + self.w4 * x[1] + self.b2
                h2 = activation(sum_h2)
                sum_o1 = self.w5 * h1 + self.w6 * h2 + self.b3
                o1 = activation(sum_o1)
                y_pred = o1

                # partial derivatives
                dL_dypred = -2 * (y_true - y_pred)

                # h1
                dh1_dw1 = x[0] * der_act(sum_h1)
                dh1_dw2 = x[1] * der_act(sum_h1)
                dh1_db1 = der_act(sum_h1)

                # h2
                dh2_dw3 = x[0] * der_act(sum_h2)
                dh2_dw4 = x[1] * der_act(sum_h2)
                dh2_db2 = der_act(sum_h2)

                # o1
                dypred_dw5 = h1 * der_act(sum_o1)
                dypred_dw6 = h2 * der_act(sum_o1)
                dypred_db3 = der_act(sum_o1)
                
                # for backprop
                dypred_dh1 = self.w5 * der_act(sum_o1)
                dypred_dh2 = self.w6 * der_act(sum_o1)

                # applying SGD to change weights and biases
                self.w1 -= learn_rate * dL_dypred * dypred_dh1 * dh1_dw1
                self.w2 -= learn_rate * dL_dypred * dypred_dh1 * dh1_dw2
                self.b1 -= learn_rate * dL_dypred * dypred_dh1 * dh1_db1

                self.w3 -= learn_rate * dL_dypred * dypred_dh2 * dh2_dw3
                self.w4 -= learn_rate * dL_dypred * dypred_dh2 * dh2_dw4
                self.b2 -= learn_rate * dL_dypred * dypred_dh2 * dh2_db2

                self.w5 -= learn_rate * dL_dypred * dypred_dw5
                self.w6 -= learn_rate * dL_dypred * dypred_dw6
                self.b3 -= learn_rate * dL_dypred * dypred_db3

                # calculating loss after avery 10 epochs
                if i % 10 == 0:
                    pred_y = np.apply_along_axis(self.ff, 1, data)
                    loss = mse(all_y_act, pred_y)
                    print(i, loss)

In [103]:
# weights (not the neural networks weight) & heights: [17, 22, 62, 49], [110, 122, 156, 145]
data = np.array([[-21, -23],
                 [-16, -11],
                 [24, 23],
                 [11, 12]])
all_y_act = np.array([1, 0, 0, 1]) # 1 is female and 0 is male
nn = new_network()
nn.train(data, all_y_act)

0 0.3009861299783996
0 0.3223210884546784
0 0.321611382465811
0 0.32164349362208405
10 0.29528993437435597
10 0.2955527901771161
10 0.29389846820825194
10 0.2944522254121016
20 0.2819795994596552
20 0.2822682072880797
20 0.28069525053092564
20 0.2813539610477761
30 0.27157915414229356
30 0.2718773880452135
30 0.2705592182472301
30 0.2712229456927708
40 0.264103405777873
40 0.26440726018097566
40 0.2633987335946388
40 0.26398604364025685
50 0.25905408127001117
50 0.25934644939501056
50 0.25861145744909014
50 0.25908929949101894
60 0.2557641345657295
60 0.2560268771731857
60 0.2555014500664862
60 0.25587216784548866
70 0.25366232401965283
70 0.25388425913400997
70 0.25351087175266657
70 0.25379070799670156
80 0.25233363448096613
80 0.2525113907545729
80 0.25224718988082323
80 0.2524541132726916
90 0.25149762225933053
90 0.2516332454357243
90 0.25144816094514105
90 0.2515976816036847
100 0.2509717719211757
100 0.2510700703015732
100 0.25094315414479507
100 0.25104768094945523
110 0.250639

In [104]:
random_1 = np.array([16, 18]) # 54, 151 --> offset: 38
random_2 = np.array([37, 26]) # 75, 159 --> offset: 133
print(nn.ff(random_1)) # female
print(nn.ff(random_2)) # male

0.47010075968504694
0.9592375447242665
